In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import os

ONNX_PATH = "models/mobilenetv3_small_best.onnx"  
IMG_PATH = "image/present4.jpeg"                        

IMG_SIZE = 224
NUM_CLASSES = 3

session = ort.InferenceSession(ONNX_PATH)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

def preprocess_image(img_path, img_size=IMG_SIZE):
    img = Image.open(img_path).convert("RGB")
    img = img.resize((img_size, img_size))
    img_array = np.array(img).astype(np.float32) / 255.0 
    img_array = np.transpose(img_array, (2, 0, 1))
    img_array = np.expand_dims(img_array, axis=0)  
    return img_array

input_tensor = preprocess_image(IMG_PATH)

outputs = session.run([output_name], {input_name: input_tensor})
preds = outputs[0] 

predicted_class = int(np.argmax(preds, axis=1)[0])
print("Predicted class:", predicted_class)
print("Raw output scores:", preds)


Predicted class: 2
Raw output scores: [[-0.14788406 -1.819021    1.7512302 ]]


In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import pandas as pd
import requests
from io import BytesIO
from tqdm import tqdm


ONNX_PATH = "models/mobilenetv3_large.onnx"
EXCEL_PATH = "data/scooter_photos.xlsx"
IMG_SIZE = 224
NUM_CLASSES = 3
N_SAMPLES = 1000

LABEL_MAP = {
    "absent": 0,
    "part": 1,
    "present": 2
}

session = ort.InferenceSession(ONNX_PATH, providers=["CPUExecutionProvider"])
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


def preprocess_image_from_url(url, img_size=IMG_SIZE):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content)).convert("RGB")
        img = img.resize((img_size, img_size))
        img_array = np.array(img).astype(np.float32) / 255.0
        img_array = np.transpose(img_array, (2, 0, 1))
        img_array = np.expand_dims(img_array, axis=0)
        return img_array
    except Exception as e:
        print(f"[Ошибка] Не удалось обработать {url}: {e}")
        return None


def predict_image(img_array):
    outputs = session.run([output_name], {input_name: img_array})
    preds = outputs[0]
    predicted_class = int(np.argmax(preds, axis=1)[0])
    return predicted_class


df = pd.read_excel(EXCEL_PATH)
df = df.iloc[:N_SAMPLES, :]
df = df.rename(columns={df.columns[2]: "scooter", df.columns[3]: "path"})

correct = 0
total = 0
per_class_correct = {0: 0, 1: 0, 2: 0}
per_class_total = {0: 0, 1: 0, 2: 0}

for _, row in tqdm(df.iterrows(), total=len(df), desc="Обработка изображений"):
    label_str = str(row["scooter"]).strip().lower()
    if label_str not in LABEL_MAP:
        continue

    true_label = LABEL_MAP[label_str]
    url = row["path"]

    img_array = preprocess_image_from_url(url)
    if img_array is None:
        continue

    pred = predict_image(img_array)

    if pred == true_label:
        correct += 1
        per_class_correct[true_label] += 1
    per_class_total[true_label] += 1
    total += 1

accuracy = correct / total * 100 if total > 0 else 0
print("\n========= РЕЗУЛЬТАТ =========")
print(f"Всего изображений: {total}")
print(f"Общая точность: {accuracy:.2f}%\n")

print("Точность по классам:")
for k, v in per_class_total.items():
    if v > 0:
        acc = per_class_correct[k] / v * 100
        print(f"Класс {k}: {acc:.2f}% ({per_class_correct[k]}/{v})")

Обработка изображений: 100%|██████████| 1000/1000 [07:58<00:00,  2.09it/s]


========= РЕЗУЛЬТАТ =========
Всего изображений: 1000
Общая точность: 85.10%

Точность по классам:
Класс 0: 96.63% (86/89)
Класс 1: 69.64% (117/168)
Класс 2: 87.21% (648/743)
